In [1]:
# !/usr/bin/env python
# coding: utf-8

In [1]:
import imp
import time
import argparse
import numpy as np
import torch

from torch_geometric.datasets import Planetoid,Reddit2,Flickr,PPI


# from torch_geometric.loader import DataLoader
# from help_funcs import prune_unrelated_edge,prune_unrelated_edge_isolated
import scipy.sparse as sp

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--debug', action='store_true',
        default=True, help='debug mode')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--seed', type=int, default=10, help='Random seed.')
parser.add_argument('--base_model', type=str, default='GCN', help='propagation model for encoder',
                    choices=['GCN','GAT','GraphSage','GIN'])
parser.add_argument('--if_smoothed', action='store_true', default=True)
parser.add_argument('--encoder_model', type=str, default='Grace', help='propagation model for encoder',
                    choices=['Grace','GraphCL'])
parser.add_argument('--dataset', type=str, default='Cora', 
                    help='Dataset',
                    choices=['Cora','Citeseer','Pubmed','PPI','Flickr','ogbn-arxiv','Reddit','Reddit2','Yelp'])
parser.add_argument('--train_lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=128,
                    help='Number of hidden units of backdoor model.')
parser.add_argument('--num_hidden', type=int, default=128,
                    help='Number of hidden units.')
parser.add_argument('--num_proj_hidden', type=int, default=128,
                    help='Number of hidden units in MLP.')
# parser.add_argument('--thrd', type=float, default=0.5)
# parser.add_argument('--target_class', type=int, default=0)
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

parser.add_argument('--test_model', type=str, default='GCN',
                    choices=['GCN','GAT','GraphSage','GIN'],
                    help='Model used to attack')
# GPU setting
parser.add_argument('--device_id', type=int, default=0,
                    help="Threshold of prunning edges")
# Contrastive Learning setting
parser.add_argument('--config', type=str, default="config.yaml")
parser.add_argument('--cl_lr', type=float, default=0.0005)
parser.add_argument('--cl_num_proj_hidden', type=int, default=128)
parser.add_argument('--cl_num_layers', type=int, default=2)
parser.add_argument('--cl_activation', type=str, default='relu')
parser.add_argument('--cl_base_model', type=str, default='GCNConv')
parser.add_argument('--cont_weight', type=float, default=1)
parser.add_argument('--add_edge_rate_1', type=float, default=0)
parser.add_argument('--add_edge_rate_2', type=float, default=0)
parser.add_argument('--drop_edge_rate_1', type=float, default=0.2)
parser.add_argument('--drop_edge_rate_2', type=float, default=0)
parser.add_argument('--drop_feat_rate_1', type=float, default=0.3)
parser.add_argument('--drop_feat_rate_2', type=float, default=0.2)
parser.add_argument('--tau', type=float, default=0.1)
parser.add_argument('--cl_num_epochs', type=int, default=200)
parser.add_argument('--cl_weight_decay', type=float, default=1e-5)
parser.add_argument('--cont_batch_size', type=int, default=0)
parser.add_argument('--noisy_level', type=float, default=0.3)
parser.add_argument('--clf_weight', type=float, default=1)
parser.add_argument('--inv_weight', type=float, default=1)
# parser.add_argument('--select_thrh', type=float, default=0.8)

# Attack
parser.add_argument('--attack', type=str, default='random',
                    choices=['nettack','random','none'],)
parser.add_argument('--select_target_ratio', type=float, default=0.1,
                    help="The number of selected target test nodes for targeted attack")
# Randomized Smoothing
parser.add_argument('--prob', default=0.2, type=float,
                    help="probability to keep the status for each binary entry")

# args = parser.parse_args()
args = parser.parse_known_args()[0]
args.cuda =  not args.no_cuda and torch.cuda.is_available()
device = torch.device(('cuda:{}' if torch.cuda.is_available() else 'cpu').format(args.device_id))

np.random.seed(args.seed)
# torch.manual_seed(args.seed)
# torch.cuda.manual_seed(args.seed)
print(args)

/tmp/ipykernel_360267/1425147217.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


Namespace(add_edge_rate_1=0, add_edge_rate_2=0, attack='random', base_model='GCN', cl_activation='relu', cl_base_model='GCNConv', cl_lr=0.0005, cl_num_epochs=200, cl_num_layers=2, cl_num_proj_hidden=128, cl_weight_decay=1e-05, clf_weight=1, config='config.yaml', cont_batch_size=0, cont_weight=1, cuda=True, dataset='Cora', debug=True, device_id=0, drop_edge_rate_1=0.2, drop_edge_rate_2=0, drop_feat_rate_1=0.3, drop_feat_rate_2=0.2, dropout=0.5, encoder_model='Grace', hidden=128, if_smoothed=True, inv_weight=1, no_cuda=False, noisy_level=0.3, num_hidden=128, num_proj_hidden=128, prob=0.2, seed=10, select_target_ratio=0.1, tau=0.1, test_model='GCN', train_lr=0.01, weight_decay=0.0005)


In [2]:
from torch_geometric.utils import to_undirected
import torch_geometric.transforms as T
transform = T.Compose([T.NormalizeFeatures()])

if(args.dataset == 'Cora' or args.dataset == 'Citeseer' or args.dataset == 'Pubmed'):
    dataset = Planetoid(root='./data/',                         name=args.dataset,                        transform=transform)
elif(args.dataset == 'Flickr'):
    dataset = Flickr(root='./data/Flickr/',                     transform=transform)
elif(args.dataset == 'Reddit2'):
    dataset = Reddit2(root='./data/Reddit2/',                     transform=transform)
elif(args.dataset == 'ogbn-arxiv'):
    from ogb.nodeproppred import PygNodePropPredDataset
    # Download and process data at './dataset/ogbg_molhiv/'
    dataset = PygNodePropPredDataset(name = 'ogbn-arxiv', root='./data/')
    split_idx = dataset.get_idx_split() 

data = dataset[0].to(device)

# from deeprobust.graph.data import Dataset, Dpr2Pyg, Pyg2Dpr
# dr_data = Dataset(root='/tmp/', name='pubmed') # load clean graph
# data = Dpr2Pyg(dr_data)
# data = data[0].to(device)

if(args.dataset == 'ogbn-arxiv'):
    nNode = data.x.shape[0]
    setattr(data,'train_mask',torch.zeros(nNode, dtype=torch.bool).to(device))
    # dataset[0].train_mask = torch.zeros(nEdge, dtype=torch.bool).to(device)
    data.val_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.test_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.y = data.y.squeeze(1)
# we build our own train test split 

In [3]:
# from utils import get_split
# # data, idx_train, idx_val, idx_clean_test, idx_atk = get_split(args,data,device)
idx_train = data.train_mask.nonzero().flatten()
idx_val = data.val_mask.nonzero().flatten()
idx_clean_test = data.test_mask.nonzero().flatten()

from torch_geometric.utils import to_undirected
from utils import subgraph
data.edge_index = to_undirected(data.edge_index)
train_edge_index,_, edge_mask = subgraph(torch.bitwise_not(data.test_mask),data.edge_index,relabel_nodes=False)
mask_edge_index = data.edge_index[:,torch.bitwise_not(edge_mask)]
# filter out the unlabeled nodes except from training nodes and testing nodes, nonzero() is to get index, flatten is to get 1-d tensor
unlabeled_idx = (torch.bitwise_not(data.test_mask)&torch.bitwise_not(data.train_mask)).nonzero().flatten()

In [4]:
# compute the label ratio
class_dict = {}
for y in data.y:
# for data in dataset:
    idx_class = y.item()
    # idx_class = data.y[0].item()
    if idx_class not in class_dict:
        class_dict[idx_class]=1
    else:
        class_dict[idx_class]+=1
print(class_dict)

{3: 818, 4: 426, 0: 351, 2: 418, 1: 217, 5: 298, 6: 180}


In [5]:
# import eval
# from models.construct import model_construct
# import copy

# model = model_construct(args,args.encoder_model,data,device)
# rs_data = copy.deepcopy(data)
# num = 10
# num_class = int(data.y.max()+1)
# idx = idx_clean_test
# counts = np.zeros(num_class,dtype=int)
# for _ in range(num):
#     # sample num different noise vectors
#     if(args.if_smoothed):
#         rs_data.edge_index,rs_data.edge_weight = model.sample_noise(data.edge_index,data.edge_weight,idx_clean_test)
#         break

In [6]:
if(args.dataset == 'Cora'):
    args.drop_edge_rate_1 = 0.2
    args.drop_edge_rate_2 = 0.4
    args.drop_feat_rate_1 = 0.3
    args.drop_feat_rate_2 = 0.4
    args.tau = 0.1
    args.cl_lr = 0.0005
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 128
elif(args.dataset == "Pubmed"):
    args.drop_edge_rate_1 = 0.4
    args.drop_edge_rate_2 = 0.1
    args.drop_feat_rate_1 = 0.0
    args.drop_feat_rate_2 = 0.2
    args.tau = 0.1
    args.cl_lr = 0.001
    args.weight_decay = 1e-5
    args.cl_num_epochs = 500
    args.num_hidden = 256

In [7]:
# from torch_geometric.utils import mask_feature,add_random_edge,dropout_adj
# from torch.distributions.bernoulli import Bernoulli
# edge_index = data.edge_index
# edge_weight = data.edge_weight
# noisy_edge_index = edge_index.clone().detach()
# if(edge_weight == None):
#     noisy_edge_weight = torch.ones([noisy_edge_index.shape[1],]).to(device)
# else:
#     noisy_edge_weight = edge_weight.clone().detach()
# # # rand_noise_data = copy.deepcopy(data)
# # rand_noise_data.edge_weight = torch.ones([rand_noise_data.edge_index.shape[1],]).to(device)
# m = Bernoulli(torch.tensor([args.prob]).to(device))
# mask_connected = m.sample(noisy_edge_weight.shape).squeeze(-1).int()
# num_edgeRemove = len((1-mask_connected).nonzero().flatten())
# num_totalEdgeState = data.x.shape[0] ** 2
# num_unconnectEdge = num_totalEdgeState - edge_index.shape[1]
# mask_unconnected = m.sample([num_unconnectEdge]).squeeze(-1).int()
# num_edgeAdd = len((1-mask_unconnected).nonzero().flatten())

# # rs = np.random.RandomState(args.seed)
# edge_index_to_add = np.random.randint(0, data.x.shape[0], (2, num_edgeAdd))
# edge_index_to_add = torch.tensor(edge_index_to_add).to(device)
# row = torch.cat([edge_index_to_add[0], edge_index_to_add[1]])
# col = torch.cat([edge_index_to_add[1],edge_index_to_add[0]])
# edge_index_to_add = torch.stack([row,col])
# updated_edge_index = torch.cat([data.edge_index,edge_index_to_add],dim=1)

# unconnected_edge_weight = torch.zeros([edge_index_to_add.shape[1],]).to(device)
# updated_edge_weight = torch.cat([noisy_edge_weight,unconnected_edge_weight],dim=0)
# rand_inputs = torch.randint_like(updated_edge_weight, low=0, high=2).squeeze().int().to(device)
# mask = m.sample(updated_edge_weight.shape).squeeze(-1).int()
# updated_edge_weight = updated_edge_weight * mask + rand_inputs * (1-mask)
# if(updated_edge_weight!=None):
#     updated_edge_index = updated_edge_index[:,updated_edge_weight.long()]
#     updated_edge_weight = torch.ones([updated_edge_index.shape[1],]).to(device)
# # updated_edge_weight = torch.ones([updated_edge_index.shape[1],]).to(device)
# # rand_inputs = torch.randint_like(updated_edge_weight, low=0, high=2).squeeze().int().to(device)

# # mask = m.sample(noisy_edge_weight.shape).squeeze(-1).int()
# # updated_edge_weight = updated_edge_weight * mask + rand_inputs * (1-mask)

In [8]:
import copy 
from models.construct import model_construct
from construct_graph import *
from models.GCN_CL import GCN_Encoder, Grace

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

from eval import label_classification,label_evaluation,label_classification_origin, smoothed_linear_evaluation

import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
# from deeprobust.graph.defense import GCN
# from deeprobust.graph.targeted_attack import Nettack
from deeprobust.graph.utils import *
# from deeprobust.graph.data import Dataset
import argparse
from tqdm import tqdm

from deeprobust.graph.data import Dataset, Pyg2Dpr, Dpr2Pyg
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack

data = data.to(device)
num_class = int(data.y.max()+1)

rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=1)

accs = []
for seed in seeds:
    # Construct and train encoder
    model = model_construct(args,args.encoder_model,data,device)
    model.fit(data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=None,verbose=True)
    # Evaluation 

=== training gcn model ===
Epoch 0, training loss: 8.596345901489258
Epoch 10, training loss: 8.578898429870605
Epoch 20, training loss: 8.498281478881836
Epoch 30, training loss: 8.281582832336426
Epoch 40, training loss: 8.140970230102539
Epoch 50, training loss: 7.665959358215332
Epoch 60, training loss: 7.43619441986084
Epoch 70, training loss: 7.423964977264404
Epoch 80, training loss: 7.174246311187744
Epoch 90, training loss: 7.06703519821167
Epoch 100, training loss: 6.903419494628906
Epoch 110, training loss: 6.896520137786865
Epoch 120, training loss: 6.927270412445068
Epoch 130, training loss: 6.675398349761963
Epoch 140, training loss: 6.586293697357178
Epoch 150, training loss: 6.575587749481201
Epoch 160, training loss: 6.456728935241699
Epoch 170, training loss: 6.447160243988037
Epoch 180, training loss: 6.360861301422119
Epoch 190, training loss: 6.2017364501953125
Epoch 200, training loss: 6.245855808258057
Epoch 210, training loss: 6.206986427307129
Epoch 220, traini

In [9]:
import models.random_smooth as random_smooth

num_class = int(data.y.max()+1)
smooth_model = random_smooth.Smooth_Ber(args, model, num_class, args.prob, data.x, if_node_level=True,device=device)
# smooth_model = random_smooth.Smooth_Ber(args, model, num_class, args.prob, data.x, data.edge_index, data.edge_weight, data.y, if_node_level=True,device=device)


prediction_distribution, acc = smooth_model._sample_noise_ber(20, data.edge_index, data.edge_weight, data.y, if_node_level = True, idx_test = idx_clean_test, idx_train = idx_train)
prediction = prediction_distribution.argmax(1)

In [10]:
classes, probs = smooth_model.certify_Ber(data.edge_index, data.edge_weight, data.y, n0=10,n=100,alpha=0.01,idx_train=idx_train, idx_test = idx_clean_test)

In [25]:
# predictions, acc = smooth_model._sample_noise_ber(10,True,idx_clean_test,idx_train)
# acc = (final_prediction==(data.y[idx_clean_test].cpu().numpy())).sum()/len(final_prediction)
# probs = np.array(probs)
# (probs > 0.5).sum()
# smooth_model.certify_K(10)
# smooth_model.certify_K(0)

# print(acc)
import math
from torch_geometric.utils import degree
for i, p_hat in enumerate(probs):
    if(p_hat==0.0):
        continue
    scale = 0.1
    # alpha = 0.01
    alpha = 1 - p_hat
    beta = 0.01
    # p_hat = 0.8
    r = scale * math.sqrt(2*math.log(1/(alpha*beta)))/(p_hat - beta)
    degrees = degree(data.edge_index[0,:])[idx_clean_test]
    print(r)
    print(r*degrees[i])

0.4154357503351642
tensor(2.0772, device='cuda:0')
0.4154357503351642
tensor(2.0772, device='cuda:0')
0.4154357503351642
tensor(2.0772, device='cuda:0')
0.4154357503351642
tensor(1.2463, device='cuda:0')
0.4154357503351642
tensor(1.6617, device='cuda:0')
0.4154357503351642
tensor(1.2463, device='cuda:0')
0.4154357503351642
tensor(1.6617, device='cuda:0')
0.4154357503351642
tensor(1.6617, device='cuda:0')
0.4154357503351642
tensor(1.6617, device='cuda:0')
0.4154357503351642
tensor(1.6617, device='cuda:0')
0.4154357503351642
tensor(1.6617, device='cuda:0')
0.4154357503351642
tensor(3.3235, device='cuda:0')
0.4154357503351642
tensor(1.2463, device='cuda:0')
0.4154357503351642
tensor(2.0772, device='cuda:0')
0.4141638646078392
tensor(2.0708, device='cuda:0')
0.4154357503351642
tensor(6.6470, device='cuda:0')
0.4154357503351642
tensor(1.2463, device='cuda:0')
0.4154357503351642
tensor(1.2463, device='cuda:0')
0.4429937820927332
tensor(2.2150, device='cuda:0')
0.4154357503351642
tensor(2.077

In [14]:
print(probs)
# acc = (((final_prediction==data.y.cpu().numpy()).sum())/len(final_prediction))
# (final_prediction==(data.y[idx_clean_test].cpu().numpy())).sum()/len(final_prediction)

[0.0, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.0, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.9354572679514184, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.808669714613969, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.0, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.0, 0.0, 0.0, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.9185880536971915, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.954992586021436, 0.0, 0.954992586021436, 0.954992586021436, 0.9354572679514184, 0.954992586021436, 0.954992586021436, 0.0, 0.954992586021436, 0.9549

In [16]:
from deeprobust.graph.data import Dataset, Pyg2Dpr, Dpr2Pyg
from torch_geometric.utils import from_scipy_sparse_matrix
from scipy.sparse import csr_matrix
from models.random_smooth import sample_noise_all_dense,sample_noise_1by1_dense
def single_test(adj, features, target_node, gcn=None):
    if gcn is None:
        # test on GCN (poisoning attack)
        gcn = GCN(nfeat=features.shape[1],
                  nhid=16,
                  nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)

        gcn = gcn.to(device)

        gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
        gcn.eval()
        output = gcn.predict()
    else:
        # test on GCN (evasion attack)
        output = gcn.predict(features, adj)
    probs = torch.exp(output[[target_node]])

    # acc_test = accuracy(output[[target_node]], labels[target_node])
    acc_test = (output.argmax(1)[target_node] == labels[target_node])
    return acc_test.item()

# Evaluation Metric: Robust Accuracy
# Node-level task
import models.random_smooth as random_smooth
args.attack='random'
print(args.attack)
if((args.dataset=='Cora') or (args.dataset=='Citeseer') or (args.dataset=='Pubmed') or (args.dataset=='Flickr') or (args.dataset=='ogbn-arxiv')):
    if(args.attack == 'nettack'):
        dpr_data = Pyg2Dpr(dataset)
        adj, features, labels = dpr_data.adj, dpr_data.features, dpr_data.labels
        features = csr_matrix(features)
        torch.manual_seed(args.seed)
        if args.cuda:
            torch.cuda.manual_seed(args.seed)
        # Train surrogate model
        surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                        nhid=16, dropout=0, with_relu=False, with_bias=False, device=device)
        surrogate = surrogate.to(device)
        surrogate.fit(features, adj, labels, idx_train, idx_val, patience=30)
        # Select target nodes
        np.random.seed(42)
        # idx = np.arange(0,adj.shape[0])
        idx = np.array(idx_clean_test.cpu())
        np.random.shuffle(idx)
        target_node_list = idx[:int(args.select_target_ratio*len(idx))]
        idx_perturn_test = torch.LongTensor(np.array(target_node_list)).to(device)
        # Conduct Attack
        # degrees = adj.sum(0).A1
        modified_adj = adj
        target_num = len(target_node_list)
        print('=== [Evasion] Attacking %s nodes respectively ===' % target_num)
        perturbation_sizes = list(range(2,5))
        for n_perturbation in perturbation_sizes:
            cnt = 0
            cl_cnt=0
            print('=== Perturbation Size %s ===' % n_perturbation)
            for target_node in tqdm(target_node_list):
                # n_perturbations = int(degrees[target_node])
                atk_model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
                atk_model = atk_model.to(device)
                atk_model.attack(features, modified_adj, labels, target_node, n_perturbation, verbose=False)
                modified_adj = atk_model.modified_adj
                modified_features = atk_model.modified_features
                acc = single_test(modified_adj, modified_features, target_node)
                print("GCN",acc)
                if acc == 0:
                    cnt += 1

                perturb_edge_index, perturb_edge_weight = from_scipy_sparse_matrix(modified_adj)
                perturb_edge_index, perturb_edge_weight = perturb_edge_index.to(device), perturb_edge_weight.to(device)
                if(args.if_smoothed):
                    perturb_edge_index,perturb_edge_weight = model.sample_noise(perturb_edge_index, perturb_edge_weight,idx_perturn_test)
                    # perturb_edge_index,perturb_edge_weight = model.sample_noise_1by1(perturb_edge_index,perturb_edge_weight,idx_perturn_test)
                z = model(data.x, perturb_edge_index,perturb_edge_weight)
                idx_target = torch.LongTensor([target_node]).to(device)
                cl_acc = label_evaluation(z, data.y, idx_train, idx_target)
                print("CL",cl_acc)
                if cl_acc == 0:
                    cl_cnt += 1
                # break
                
            print('[GCN] misclassification rate : %s' % (cnt/target_num))
            print('[CL] misclassification rate : %s' % (cl_cnt/target_num))
    elif(args.attack == 'random'):
        import construct_graph
        perturbation_sizes = list(range(1,30))
        for n_perturbation in perturbation_sizes:
            noisy_data = copy.deepcopy(data)
            for idx in idx_clean_test:
                noisy_data = construct_graph.generate_node_noisy(args,noisy_data,idx,n_perturbation,device)
                noisy_data = noisy_data.to(device)
            model.eval()
            import models.random_smooth as random_smooth

            num_class = int(data.y.max()+1)
            smooth_model = random_smooth.Smooth_Ber(args, model, num_class, args.prob, noisy_data.x, if_node_level=True,device=device)
            # smooth_model = random_smooth.Smooth_Ber(args, model, num_class, args.prob, data.x, data.edge_index, data.edge_weight, data.y, if_node_level=True,device=device)
            # classes, probs = smooth_model.certify_Ber(n0=10,n=100,alpha=0.01,idx_train=idx_train, idx_test = idx_clean_test)

            prediction_distribution, acc = smooth_model._sample_noise_ber(20, noisy_data.edge_index, noisy_data.edge_weight, noisy_data.y, if_node_level = True, idx_test = idx_clean_test, idx_train = idx_train)
            prediction = prediction_distribution.argmax(1)
            # acc, prediction = smoothed_linear_evaluation(args, model, noisy_data.x, noisy_data.edge_index, noisy_data.edge_weight, 50, noisy_data.y, idx_train, idx_clean_test, device)
            
            
            # if(args.if_smoothed):
            #     noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise(noisy_data.edge_index,noisy_data.edge_weight,idx_clean_test)
            #     # noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise_1by1(noisy_data.edge_index,noisy_data.edge_weight,idx_clean_test)
            #     # edge_index_1,edge_weight_1 = model.sample_noise_all_sparse(args,edge_index_1,edge_weight_1,self.features)
            # z = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)
            # acc = label_evaluation(z, noisy_data.y, idx_train, idx_clean_test)
            print(acc)
    elif(args.attack == 'none'):
        model.eval()
        rs_data = copy.deepcopy(data)
        if(args.if_smoothed):
            # rs_data.edge_index,rs_data.edge_weight = sample_noise_1by1_dense(args,data.edge_index,data.edge_weight,idx_clean_test,device)
            # rs_data.edge_index,rs_data.edge_weight = model.sample_noise_1by1(data.edge_index,data.edge_weight,idx_clean_test)
            rs_data.edge_index,rs_data.edge_weight = model.sample_noise(data.edge_index,data.edge_weight,idx_clean_test)
        z = model(rs_data.x, rs_data.edge_index,rs_data.edge_weight)
        acc = label_evaluation(z, rs_data.y, idx_train, idx_clean_test)
        print(acc)
        # accs.append(acc)

random
0.806
0.806
0.793
0.793
0.784
0.775
0.771
0.768
0.758
0.757
0.755
0.74
0.739
0.708
0.705
0.744
0.744
0.734
0.726
0.734
0.727
0.718
0.705
0.714
0.709
0.7
0.695
0.692
0.708


In [33]:
import models.random_smooth as random_smooth
# counts = np.zeros(7,dtype=int)
# for i in range(10):
#     prediction = np.array([1,])
#     counts[prediction]+=1
# counts
# Number of samples to collect
import eval
def _sample_noise_ber(idx,num,edge_index,edge_weight):
    counts = np.zeros(num_class,dtype=int)
    for _ in range(num):
        # sample num different noise vectors
        rs_data = copy.deepcopy(data)
        if(args.if_smoothed):
            rs_data.edge_index,rs_data.edge_weight = model.sample_noise(edge_index,edge_weight,torch.tensor([idx]))
        z = model(rs_data.x, rs_data.edge_index,rs_data.edge_weight)
        acc, prediction = eval.linear_evaluation(z, rs_data.y, idx_train, torch.tensor([i]))
        # print(acc,prediction)
        counts[prediction]+=1
    return counts

cnt = 0
cnt_certify = 0

alpha = 0.001
n0 = 10
n=100
v_range = [0,21]
num_class = int(data.y.max()+1)
idx = idx_clean_test
labels = data.y
for i in idx_clean_test[2:]:
    counts_selection = _sample_noise_ber(i,n0,data.edge_index,data.edge_weight)
    cAHat = counts_selection.argmax().item()    # prediction
    print(counts_selection)
    counts_estimation = _sample_noise_ber(i,n,data.edge_index,data.edge_weight)
    print(counts_estimation)
    nA = counts_estimation[cAHat].item()
    pABar = random_smooth._lower_confidence_bound(nA,n,alpha)
    print(pABar)
    correct = int(cAHat == labels[i])
    cnt += 1
    cnt_certify += correct
    print("correct",correct)
    if(correct):
        v = certify_K(pABar, alpha, 2708, v_range, 'cora')
    print("certify_K:{}".format(v))
    # break
print("certify acc:", float(cnt_certify) / cnt)


[0 1 9 0 0 0 0]
[ 0  8 89  3  0  0  0]
0.7623693074964116
correct 1
0.76236930749641162752 0.23763069250358837248


ZeroDivisionError: 0.0 cannot be raised to a negative power

In [28]:
import models.random_smooth as random_smooth
pABar = random_smooth._lower_confidence_bound(10,100,alpha)
pABars = [0.09450548934688922, 0.8927598671315771, 0.6421246907768379, 0.8299983991836403, 0.8446326941895317, 0.28098366650865014]
rs_data = copy.deepcopy(data)
rs_data.edge_index,rs_data.edge_weight = model.sample_noise(rs_data.edge_index,rs_data.edge_weight,idx_clean_test)
z = model(rs_data.x, rs_data.edge_index,rs_data.edge_weight)
acc, prediction = eval.linear_evaluation(z, rs_data.y, idx_train, idx_clean_test)
accurate = (prediction == rs_data.y[idx_clean_test].cpu().numpy())
# print(prediction,rs_data.y[idx_clean_test].cpu().numpy())
# print(accurate)
v_range = [0,21]
K = np.zeros(21, dtype=int)
# from certify_K import certify_K
for idx in range(len(idx_clean_test)):
	if accurate[idx]:
		print(idx)
		v = certify_K(pABars[idx], alpha, 2708, v_range, 'cora')
		print('pAHat:', pABars[idx], 'Certified K:', v)
		K[v] += 1
print(K)

0
1 1 0 0
1 2708 1 1
1 3665278 2 2
1 3306080756 3 3
1 2235737111245 4 4
1 1209086629761296 5 5
1 544693526707463848 6 6
1 210251701309081045328 7 7
1 70986230654478487928866 8 8
1 21295869196343546378659800 9 9
1 5747755096093123167600280020 10 10
1 1409767568114476936925959590360 11 11
1 316845260933728691574109417933410 12 12
1 65708832575179427114138383903728720 13 13
1 12648950270722039719471638901467778600 14 14
1 2271751468621678333617106346703613036560 15 15
1 382364169062386234526929211979551869216005 16 16
1 60548490771526102549793731685232566584087380 17 17
1 9051999370343152331194162886942268704321063310 18 18
1 1281572542432793672153278850835510674453876858100 19 19
1 172307428330089109221008341494834410180323743571545 20 20
1 22055350826251405980289067711338804503081439177157760 21 21
1 2693760348642614903138032951834880349989992139501040960 22 22
1 314584360715394070862119848201238635655352995073904174720 23 23
1 35194125355034711677699658017513572363942616323893029546800

ZeroDivisionError: 0.0 cannot be raised to a negative power

In [30]:
import numpy as np
#from tqdm import trange
import argparse
from decimal import Decimal

# parser = argparse.ArgumentParser(description='Get thresholds')
# parser.add_argument('--r_start', default=0, type=int, 
#                     help='[r_start, r_end)')
# parser.add_argument('--r_end', default=21, type=int, 
#                     help='[r_start, r_end)')
# parser.add_argument('--a', type=int, default=80,
#                     help='alpha = a / 100')
# args = parser.parse_args()

# v_range = [args.r_start, args.r_end]

# fn = 'cora'
# global_d =  2708 # cora

# # fn = 'citeseer'
# # global_d = 3327 # citeseer

# # fn = 'pubmed'
# # global_d = 19717 # pubmed


def certify_K(p_l, frac_alpha, global_d, v_range, fn):

	frac_beta = (1 - frac_alpha)
	
	alpha = int(frac_alpha * 100)
	beta = 100 - alpha

	for v in range(v_range[0], v_range[1]):

		plower_Z = int(p_l * 100 ** 10) * (100 ** (global_d-10))
		pupper_Z = int((1-p_l) * 100 ** 10) * (100 ** (global_d-10))
		total_Z = 100 ** global_d

		complete_cnt = []
		cnt = np.load('./list_counts/{}/complete_count_{}.npy'.format(fn, v), allow_pickle=True)
		complete_cnt += list(cnt)
		
		raw_cnt = 0
		
		outcome = []
		for ((s, t), c) in complete_cnt:
			outcome.append((
				# likelihood ratio x flips s, x bar flips t
				# and then count, s, t
				(alpha ** (t - s)) * (beta ** (s - t)), c, s, t
			))
			if s != t:	
				outcome.append((
					(alpha ** (s - t)) * (beta ** (t - s)), c, t, s
				))

			raw_cnt += c
			if s != t:
				raw_cnt += c

		# sort likelihood ratio in a descending order, i.e., r1 >= r2 >= ...
		outcome_descend = sorted(outcome, key = lambda x: -x[0])
		p_given_lower = 0
		q_given_lower = 0
		for i in range(len(outcome_descend)):
			ratio, cnt, s, t = outcome_descend[i]
			p = (alpha ** (global_d - s)) * (beta ** s)
			q = (alpha ** (global_d - t)) * (beta ** t)
			q_delta_lower = q * cnt
			p_delta_lower = p * cnt

			if p_given_lower + p_delta_lower < plower_Z:
				p_given_lower += p_delta_lower
				q_given_lower += q_delta_lower
			else:
				q_given_lower += (plower_Z - p_given_lower) / Decimal(ratio)
				#q_given_lower += q * (plower_Z - p_given_lower) / Decimal(p)
				break
		q_given_lower /= total_Z


		# sort likelihood ratio in a ascending order
		outcome_ascend = sorted(outcome, key = lambda x: x[0])
		p_given_upper = 0
		q_given_upper = 0
		for i in range(len(outcome_ascend)):
			ratio, cnt, s, t = outcome_ascend[i]
			p = (alpha ** (global_d - s)) * (beta ** s)
			q = (alpha ** (global_d - t)) * (beta ** t)
			q_delta_upper = q * cnt
			p_delta_upper = p * cnt

			if p_given_upper + p_delta_upper < pupper_Z:
				p_given_upper += p_delta_upper
				q_given_upper += q_delta_upper
			else:
				q_given_upper += (pupper_Z - p_given_upper) / Decimal(ratio)
				#q_given_upper += q * (pupper_Z - p_given_upper) / Decimal(p)
				break
		q_given_upper /= total_Z

		print(q_given_lower, q_given_upper)

		if q_given_lower - q_given_upper < 0:
			return v


In [9]:
model = model_construct(args,'GCN',data,device)
model.fit(data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=200,verbose=True)
model.test(data.x,data.edge_index,data.edge_weight,data.y,idx_clean_test)

=== training gcn model ===
Epoch 0, training loss: 1.9459470510482788
acc_val: 0.5640
Epoch 10, training loss: 1.5109736919403076
acc_val: 0.7180
Epoch 20, training loss: 0.7374580502510071
acc_val: 0.7840
Epoch 30, training loss: 0.2591830790042877
acc_val: 0.7940
Epoch 40, training loss: 0.09540168941020966
acc_val: 0.7840
Epoch 50, training loss: 0.052881985902786255
acc_val: 0.7760
Epoch 60, training loss: 0.028508443385362625
acc_val: 0.7840
Epoch 70, training loss: 0.024615274742245674
acc_val: 0.7840
Epoch 80, training loss: 0.016384324058890343
acc_val: 0.7840
Epoch 90, training loss: 0.017661025747656822
acc_val: 0.7840
Epoch 100, training loss: 0.01655925065279007
acc_val: 0.7880
Epoch 110, training loss: 0.014042257331311703
acc_val: 0.7860
Epoch 120, training loss: 0.015547742135822773
acc_val: 0.7880
Epoch 130, training loss: 0.012804940342903137
acc_val: 0.7840
Epoch 140, training loss: 0.013380806893110275
acc_val: 0.7880
Epoch 150, training loss: 0.013110270723700523
ac

0.803

In [11]:
from deeprobust.graph.data import Dataset, Pyg2Dpr, Dpr2Pyg
from torch_geometric.utils import from_scipy_sparse_matrix
from scipy.sparse import csr_matrix
def single_test(adj, features, target_node, gcn=None):
    if gcn is None:
        # test on GCN (poisoning attack)
        gcn = GCN(nfeat=features.shape[1],
                nhid=16,
                nclass=labels.max().item() + 1,
                dropout=0.5, device=device)

        gcn = gcn.to(device)

        gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
        gcn.eval()
        output = gcn.predict()
    else:
        # test on GCN (evasion attack)
        output = gcn.predict(features, adj)
    probs = torch.exp(output[[target_node]])

    # acc_test = accuracy(output[[target_node]], labels[target_node])
    acc_test = (output.argmax(1)[target_node] == labels[target_node])
    return acc_test.item()

# Evaluation Metric: Robust Accuracy
# Node-level task
print(args.attack)
if((args.dataset=='Cora') or (args.dataset=='Citeseer') or (args.dataset=='Pubmed') or (args.dataset=='Flickr') or (args.dataset=='ogbn-arxiv')):
    if(args.attack == 'nettack'):
        dpr_data = Pyg2Dpr(dataset)
        adj, features, labels = dpr_data.adj, dpr_data.features, dpr_data.labels
        features = csr_matrix(features)
        torch.manual_seed(args.seed)
        if args.cuda:
            torch.cuda.manual_seed(args.seed)
        # Train surrogate model
        surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                        nhid=16, dropout=0, with_relu=False, with_bias=False, device=device)
        surrogate = surrogate.to(device)
        surrogate.fit(features, adj, labels, idx_train, idx_val, patience=30)
        # Select target nodes
        np.random.seed(42)
        # idx = np.arange(0,adj.shape[0])
        idx = np.array(idx_clean_test.cpu())
        np.random.shuffle(idx)
        target_node_list = idx[:int(args.select_target_ratio*len(idx))]
        idx_perturn_test = torch.LongTensor(np.array(target_node_list)).to(device)
        # Conduct Attack
        # degrees = adj.sum(0).A1
        modified_adj = adj
        target_num = len(target_node_list)
        print('=== [Evasion] Attacking %s nodes respectively ===' % target_num)
        perturbation_sizes = list(range(1,11))
        for n_perturbation in perturbation_sizes:
            cnt = 0
            cl_cnt=0
            print('=== Perturbation Size %s ===' % n_perturbation)
            for target_node in tqdm(target_node_list):
                # n_perturbations = int(degrees[target_node])
                atk_model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
                atk_model = atk_model.to(device)
                atk_model.attack(features, modified_adj, labels, target_node, n_perturbation, verbose=False)
                modified_adj = atk_model.modified_adj
                modified_features = atk_model.modified_features
                acc = single_test(modified_adj, modified_features, target_node)
                print("GCN",acc)
                if acc == 0:
                    cnt += 1

                perturb_edge_index, perturb_edge_weight = from_scipy_sparse_matrix(modified_adj)
                perturb_edge_index, perturb_edge_weight = perturb_edge_index.to(device), perturb_edge_weight.to(torch.float).to(device)
                idx_target = torch.LongTensor([target_node]).to(device)
                cl_acc = model.test(data.x,perturb_edge_index,perturb_edge_weight,data.y,idx_target)
                cl_acc = int(cl_acc)
                # if(args.if_smoothed):
                #     # perturb_edge_index,perturb_edge_weight = sample_noise_1by1_dense(args,perturb_edge_index, perturb_edge_weight,idx_perturn_test, device)
                #     perturb_edge_index,perturb_edge_weight = model.sample_noise(perturb_edge_index, perturb_edge_weight,idx_perturn_test)
                #     # perturb_edge_index,perturb_edge_weight = model.sample_noise_1by1(perturb_edge_index, perturb_edge_weight,idx_perturn_test)
                # z = model(data.x, perturb_edge_index,perturb_edge_weight)
                # idx_target = torch.LongTensor([target_node]).to(device)
                # cl_acc = label_evaluation(z, data.y, idx_train, idx_target)
                print("Robust GNN",cl_acc)
                if cl_acc == 0:
                    cl_cnt += 1
                # break
            print('[GCN] misclassification rate : %s' % (cnt/target_num))
            print('[CL] misclassification rate : %s' % (cl_cnt/target_num))
    elif(args.attack == 'random'):
        import construct_graph
        import copy
        perturbation_sizes = list(range(0,21))
        for n_perturbation in perturbation_sizes:
            print("Perturbation Size:{}".format(n_perturbation))
            noisy_data = copy.deepcopy(data)
            if(n_perturbation > 0):
                for idx in idx_clean_test:
                    noisy_data = construct_graph.generate_node_noisy(args,noisy_data,idx,n_perturbation,device)
                    noisy_data = noisy_data.to(device)
            model.eval()
            acc = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
            # if(args.if_smoothed):
            #     noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise(noisy_data.edge_index,noisy_data.edge_weight,idx_clean_test)
            #     # noisy_data.edge_index,noisy_data.edge_weight = sample_noise_1by1_dense(args,noisy_data.edge_index,noisy_data.edge_weight,idx_clean_test, device)
            #     # noisy_data.edge_index,noisy_data.edge_weight = model.sample_noise_1by1(noisy_data.edge_index,noisy_data.edge_weight,idx_clean_test)
            # z = model(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight)
            # acc = label_evaluation(z, noisy_data.y, idx_train, idx_clean_test)
            print("Accuracy:",acc)
    elif(args.attack == 'none'):
        model.eval()
        # rs_data = copy.deepcopy(data)
        acc = model.test(data.x,data.edge_index,data.edge_weight,data.y,idx_clean_test)
        # if(args.if_smoothed):
        #     rs_data.edge_index,rs_data.edge_weight = model.sample_noise(data.edge_index,data.edge_weight,idx_clean_test)
        #     # rs_edge_index, rs_edge_weight = sample_noise_1by1_dense(args,data.edge_index,data.edge_weight,idx_clean_test, device)
        #     # rs_data.edge_index,rs_data.edge_weight = model.sample_noise_1by1(data.edge_index,data.edge_weight,idx_clean_test)
        # z = model(rs_data.x, rs_data.edge_index,rs_data.edge_weight)
        # acc = label_evaluation(z, rs_data.y, idx_train, idx_clean_test)
        print("Accuracy:",acc)
        # accs.append(acc)

random
Perturbation Size:0
Accuracy: 0.803
Perturbation Size:1
Accuracy: 0.8190000000000001
Perturbation Size:2
Accuracy: 0.8230000000000001
Perturbation Size:3
Accuracy: 0.81
Perturbation Size:4
Accuracy: 0.79
Perturbation Size:5
Accuracy: 0.781
Perturbation Size:6
Accuracy: 0.767
Perturbation Size:7
Accuracy: 0.757
Perturbation Size:8
Accuracy: 0.751
Perturbation Size:9
Accuracy: 0.736
Perturbation Size:10
Accuracy: 0.732
Perturbation Size:11
Accuracy: 0.715
Perturbation Size:12
Accuracy: 0.706
Perturbation Size:13
Accuracy: 0.6880000000000001
Perturbation Size:14
Accuracy: 0.679
Perturbation Size:15
Accuracy: 0.665
Perturbation Size:16
Accuracy: 0.648
Perturbation Size:17
Accuracy: 0.641
Perturbation Size:18
Accuracy: 0.62
Perturbation Size:19
Accuracy: 0.613
Perturbation Size:20
Accuracy: 0.613
